In [1]:
!python -V

Python 3.9.21


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

from sklearn.metrics import root_mean_squared_error

In [6]:
import mlflow

mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1749414543793, experiment_id='1', last_update_time=1749414543793, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [17]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
        print(f"Number of records loaded: {len(df)}")

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    return df

In [18]:
df_train = read_dataframe('./data/yellow_tripdata_2023-03.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

Number of records loaded: 3403766
Number of records loaded: 2913955


In [19]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [20]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [21]:
import xgboost as xgb

In [22]:
from pathlib import Path

In [23]:
models_folder = Path('models')
models_folder.mkdir(exist_ok=True)

In [24]:
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=30,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/ananya/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:23:15] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.33640
[1]	validation-rmse:8.68529
[2]	validation-rmse:8.11660
[3]	validation-rmse:7.62189
[4]	validation-rmse:7.19409
[5]	validation-rmse:6.82614
[6]	validation-rmse:6.51076
[7]	validation-rmse:6.24220
[8]	validation-rmse:6.01494
[9]	validation-rmse:5.82356
[10]	validation-rmse:5.66261
[11]	validation-rmse:5.52832
[12]	validation-rmse:5.41680
[13]	validation-rmse:5.32372
[14]	validation-rmse:5.24691
[15]	validation-rmse:5.18322
[16]	validation-rmse:5.13087
[17]	validation-rmse:5.08760
[18]	validation-rmse:5.05168
[19]	validation-rmse:5.02186
[20]	validation-rmse:4.99695
[21]	validation-rmse:4.97683
[22]	validation-rmse:4.96006
[23]	validation-rmse:4.94583
[24]	validation-rmse:4.93421
[25]	validation-rmse:4.92407
[26]	validation-rmse:4.91582
[27]	validation-rmse:4.90896
[28]	validation-rmse:4.90308
[29]	validation-rmse:4.89811


/home/ananya/.local/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [23:27:32] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/06/08 23:27:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
